<a href="https://colab.research.google.com/github/the-full-stack/fsdl-text-recognizer-2022/blob/main/notebooks/lab99_colab_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://fsdl.me/logo-720-dark-horizontal">

# Testing the Colab Environment

Run this notebook to test that the Colab environment is working:
Runtime > Run all in the toolbar.

Note that the final cell requires you to provide a secret interactively.

In [ ]:
import sys

in_colab = "google.colab" in sys.modules
repo = "fsdl-text-recognizer-2022"

assert in_colab

!git clone https://github.com/full-stack-deep-learning/{repo}
%cd /content/{repo}

## Core Package Version Checking

The Colab environment has no guarantees of stability and no release announcements,
so every once in a while, I run this set of cells to confirm all core package versions are as expected.

For example, this notebook detected a change to the PyTorch and cuDNN libraries on July 12, 2022,
[14 days after PyTorch 1.12 was released](https://github.com/pytorch/pytorch/releases/tag/v1.12.0).

In [ ]:
python_version = !python --version
assert 'Python 3.10.' in python_version[0], python_version[0]
python_version[0]

In [ ]:
pip_version = !pip --version
assert 'pip 23.' in pip_version[0], pip_version[0]
pip_version[0]

In [ ]:
!pip list | grep setuptools

In [ ]:
!pip list | grep torch

In [ ]:
import torch

assert "2.1." in torch.__version__, torch.__version__
assert "cu118" in torch.__version__, torch.__version__
torch.__version__

In [ ]:
assert torch.version.cuda == "11.8", torch.version.cuda
torch.version.cuda

In [ ]:
cudnn_version = ".".join(str(v) for v in torch._C._cudnn.getRuntimeVersion())
assert "8.7." in cudnn_version, cudnn_version
cudnn_version

In [ ]:
import torchvision

assert "0.16." in torchvision.__version__, torchvision.__version__
assert "cu118" in torchvision.__version__, torchvision.__version__
torchvision.__version__

In [ ]:
import PIL

assert "9.4." in PIL.__version__, PIL.__version__
PIL.__version__

In [ ]:
import jinja2

assert "3.1" in jinja2.__version__, jinja2.__version__
jinja2.__version__

## Installation Checks

We keep versions aligned in part so that there isn't a slow installation phase whenver the Colab runtime spins up.

These cells install the requirements and times the installation.
It should be no more than two minutes total.

In [ ]:
%%writefile requirements/prod.in
h5py
importlib-metadata>=4.4
numpy
pyngrok
requests
smart_open[s3]
tqdm
# versioned for stability
gantry==0.4.9
gradio==3.0.21
# versioned to match Google Colab up to minor
Jinja2>=3.1,<3.2
pillow>=9.4,<9.5
torch>=2.1,<2.2
torchvision>=0.16,<0.17
# versioned to avoid breaking change in minor version update
markupsafe<2.1

In [ ]:
from time import time as now


start_prod_install = now()
!pip install -r requirements/prod.in
prod_install_duration = now() - start_prod_install

In [ ]:
prod_install_duration

In [ ]:
import seaborn as sns

assert "0.12" in sns.__version__, sns.__version__

sns.__version__

In [ ]:
import tornado

assert "6.3" in tornado.version, tornado.version

tornado.version

In [ ]:
%%writefile requirements/dev.in
-c prod.txt
boltons
coverage[toml]
defusedxml
itermplot
ipywidgets
matplotlib
notebook
nltk
pre-commit
pytest
pytest-cov
scipy
toml
# versioned to give pip hints
coverage[toml]==6.4
pytest==7.1.1
pytest-cov==3.0.0
# versioned to match Google Colab
seaborn>=0.12,<0.13
tornado>=6.3,<6.4
# versioned to improve stability
pytorch-lightning==1.6.3
torchmetrics<0.8
wandb==0.12.17

In [ ]:
start_dev_install = now()
!sed 1d requirements/dev.in | grep -v "#" | xargs pip install
dev_install_duration = now() - start_dev_install

In [ ]:
total_install_duration = prod_install_duration + dev_install_duration
print(f"{total_install_duration}")

maximum_allowed_time = 120  # seconds

assert total_install_duration < maximum_allowed_time, f"install duration was too long: {int(total_install_duration)}"

In [ ]:
import pytorch_lightning as pl  # checking that we have our dev libraries

Then we need to do Python path handling...

In [ ]:
pythonpath = !echo $PYTHONPATH
if "." not in pythonpath[-1]:
  pythonpath = ["."] + pythonpath
  %env PYTHONPATH={":".join(pythonpath)}
!echo $PYTHONPATH

...and turn on "hot" reloading of modules...

In [ ]:
%load_ext autoreload
%autoreload 2

...so that we can import our development libraries and edit them on the fly.

In [ ]:
import text_recognizer
import training

## Run Tests

And then we run our tests to check that the code is working.

The below tests should take about 10 minutes,
depending on the GPU you receive from Colab.

Note that the tests require a W&B API key so that we can download
the current production model
-- either set it interactively after running the cell below
or set it as an environment variable,
`$WANDB_API_KEY`.

In [ ]:
! wandb login

out = !./tasks/unit_test.sh && echo "success" || echo "failure"

if out[-1] == "failure":
    raise RuntimeError("\n".join(out[:-1]))
print(out[-1])